In [153]:
def char2bit(string):
    """returns list of lists of 8-bit code for input string"""
    bits = []
    for char in string:
        bits = bits + list(("0"+bin(ord(char))[2:]))
    return bits

def pad(bits):
    """pad with zeros to (multiple of 512)-64"""
    multiplier=(len(bits)+64)//512+1
    pad_length = multiplier*512 -64
    bits = bits + list((pad_length-len(bits))*"0")
    return bits
def bit2hex(bit):
    """outputs hex representation of given bit sequence"""
    #python(hex function)

def prime_numbers(n):
    primes=[]
    number=2
    while len(primes)<n:
        divisor_found=False
        for divisor in range(2, number):
            divisor_found = number%divisor==0
            if divisor_found == True:
                break
        if divisor_found==False:
            primes.append(number)
        number+=1
    return primes

def initialize_constants(primes, root_dim, length):
    constants=[]
    for prime in primes:
        root = prime**(1/root_dim)
        fraction = root - int(root)
        constants.append(hex(int(fraction*2**length)))
    return constants

def chunker(bits, n):
    chunks=[]
    for i in range(n,len(bits)+1,n):
        chunks.append(bits[i-n:i])
    return chunks

def xor(bit1, bit2):
    bits = []
    for i in range(len(bit1)):
        if bit1[i]!=bit2[i]:
            bits.append("1")
        else:
            bits.append("0")
    return bits

def rotate_right(bits, n):
    """rotate right by n values"""
    for step in range(n):
        bits = [bits[-1]]+bits[:-1]
    return bits

def shift_right(bits, n):
    """shift right by n values (fill with zeros)"""
    for step in range(n):
        bits = ["0"]+bits[:-1]
    return bits

def majority(bits):
    """given a list of bits: returns majority for each bit"""
    pass

def add(bit1, bit2):
    """bitwise addition"""
    pass

def sigma0(bits):
    """"""
    pass
def sigma1(bits):
    pass

In [115]:
string = "hello world"
#1. Preprocess
#1.1 convert to binary
bits = char2bit(string)
message_length = list(bin(len(bits)))[2:] #needed later
#1.2 append a single one
bits = bits+["1"]
#1.3 pad with zeros to (multiple of 512) - 64
bits = pad(bits)
#1.4 add 64 bits with length of initial message
message_length = list((64-len(message_length))*"0") + message_length
bits = bits + message_length

In [116]:
#2. initialize hash values h
h_primes = prime_numbers(8)
h_constants = initialize_constants(h_primes, root_dim=2, length=32)

In [117]:
#3. Initialize constants k
k_primes=prime_numbers(64)
k_constants = initialize_constants(k_primes,root_dim=3, length=32)

In [121]:
#4. split in chunks of 512
chunks= chunker(bits, 512)
for message in chunks:
    #5.Create message schedule
    #5.1 split in chunks of 32 bits
    words = chunker(message, 32)
    #5.2 add 48 words initialized to zero
    for i in range(48):
        words.append(list("0"*32))
    #5.3 modify zero indices
    for i in range(16,64):
        s0= xor(rotate_right(words[i-15],7), xor(rotate_right(words[i-15],18), shift_right(words[i-15],3)))
        s1=xor(rotate_right(words[i-2],17), xor(rotate_right(words[i-2],19), shift_right(words[i-2],10)))
        words[i] = words[i-16]+s0+words[i-7]+s1

64


In [156]:
a=["0","0","1"]
b=["1","1","1"]
c=["0","1","0"]

result=[]
carryover="0"
for i in range(len(a)-1,-1,-1):
    result[x] = xor(a, xor(b,carryover))
    c = 